<a href="https://colab.research.google.com/github/abequette/CS167Fall22Code/blob/main/Day21_Notes_ANNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 21 Code: Artificial Neural Networks

We're going to start off by using sklearn MLP to implement a multilayer perceptron, and then we're going to use a deep learning framework, Tensorflow with Keras to build a neural network. 

In [1]:
from google.colab import drive
import pandas
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data = pandas.read_csv('/content/drive/MyDrive/CS167/irisData.csv')
data.head()

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
import pandas
import numpy
from sklearn.model_selection import train_test_split

#Split the dataset
predictors = data.columns.drop('species')
target = "species"
train_data, test_data, train_sln, test_sln = train_test_split(data[predictors], data[target], test_size = 0.2, random_state=41)

#Normalize Data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_data)
train_data_norm = scaler.transform(train_data)
test_data_norm = scaler.transform(test_data)


## Build out a Multilayer Perceptron using Scikit-Learn:
Here are the links to the documentation: 
- [sklearn.neural_network.MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)
- [sklearn.neural_network.MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)


In [4]:
# Set up MLP
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix

mlp = MLPClassifier(random_state=0,hidden_layer_sizes = (100,), max_iter = 800)
mlp.fit(train_data_norm,train_sln)
predictions = mlp.predict(test_data_norm)

print("Accuracy: ", metrics.accuracy_score(test_sln,predictions))

# Confusion Matrix
vals = data[target].unique() ## possible classification values (species)
conf_mat = metrics.confusion_matrix(test_sln, predictions, labels=vals)
print(pandas.DataFrame(conf_mat, index = "True " + vals, columns = "Pre " + vals))

Accuracy:  0.9
                      Pre Iris-setosa  Pre Iris-versicolor  Pre Iris-virginica
True Iris-setosa                    8                    1                   0
True Iris-versicolor                0                   10                   1
True Iris-virginica                 0                    1                   9


## In-Class Exercise:

1. Read in the Boston Housing dataset
2. Normalize your data
3. Use a [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) to predict the price of a house 'MEDV'
4. Play around with changing the parameters, see what the best R2 score you can get is. 


In [6]:
# Your code goes here for the In-Class Exercise
# 1. Read in the Boston Housing dataset
import pandas
housing_data = pandas.read_csv('/content/drive/MyDrive/CS167/HousingData.csv') 

# clean the data
housing_data['CRIM'].fillna(housing_data['CRIM'].mean(),inplace=True)
housing_data['ZN'].fillna(housing_data['ZN'].mean(),inplace=True)
housing_data['INDUS'].fillna(housing_data['INDUS'].mean(),inplace=True)
housing_data['CHAS'].fillna(housing_data['CHAS'].mean(),inplace=True)
housing_data['AGE'].fillna(housing_data['AGE'].mean(),inplace=True)
housing_data['LSTAT'].fillna(housing_data['LSTAT'].mean(),inplace=True)

#Split the dataset
predictors = housing_data.columns.drop('MEDV')
target = "MEDV"
train_data, test_data, train_sln, test_sln = train_test_split(housing_data[predictors], housing_data[target], test_size = 0.2, random_state=0)



In [7]:
#2. Normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_data)
train_data_normalized = scaler.transform(train_data)
test_data_normalized = scaler.transform(test_data)

In [8]:
#3. Use a MLPRegressor to predict the price of a house 'MEDV'
from sklearn.neural_network import MLPRegressor
from sklearn import metrics

mlp = MLPRegressor(random_state=0)
mlp.fit(train_data_normalized,train_sln)
predictions = mlp.predict(test_data_normalized)

print("MLP Regression R2:", metrics.r2_score(test_sln, predictions))


MLP Regression R2: 0.49660446847147843


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [21]:
#4. Play around with the parameters
mlp = MLPRegressor(random_state=0, hidden_layer_sizes=(100,), max_iter = 500)
mlp.fit(train_data_normalized,train_sln)
predictions = mlp.predict(test_data_normalized)

print("MLP Regression R2:", metrics.r2_score(test_sln, predictions))

MLP Regression R2: 0.693869049915123


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


# Introducing Deep Learning Frameworks

Go ahead and go up to 'Runtime', and select 'change runtime type' from the dropdown list, select 'GPU'. If you complete this step correctly, the following code should say `Found GPU at: /device:GPU:0` or something similar.

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Iris Dataset with Keras

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

from keras.models import Sequential
from keras.layers import Dense

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy

# we're going to use the iris dataset, but load it from sklearn 
iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

# One hot encoding
enc = OneHotEncoder()
Y = enc.fit_transform(y[:, numpy.newaxis]).toarray()

# Split the data set into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

#normalize the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)



In [3]:
# display a row of data
print(X_train_norm[0,:])
print(Y_train[0,:])

[ 0.37346331 -0.58519388  0.54075378  0.74234434]
[0. 0. 1.]


In [4]:
#build our neural network model
n_features = X.shape[1] #X generally stands for our predictors
n_classes = Y.shape[1] #Y generally stands for our target

model = Sequential(name='iris_1')
model.add(Dense(2, input_dim=n_features, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

from sklearn.metrics import r2_score

# Compile model
model.compile(loss='mean_squared_error',
              optimizer='sgd', 
              metrics=['accuracy'])
model.summary()

Model: "iris_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 10        
                                                                 
 dense_1 (Dense)             (None, 2)                 6         
                                                                 
 dense_2 (Dense)             (None, 3)                 9         
                                                                 
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


[Keras Model Training APIs](https://keras.io/api/models/model_training_apis/)


In [5]:
# train the model
model.fit(X_train_norm, Y_train, batch_size=15, epochs=5) #add verbose = 0 to make output minimal

# cross-validation; make predictions and get error
print("----"*30)
mse, acc =  model.evaluate(X_test_norm, Y_test)
print('Mean Squared Error:',mse)
print('Test accuracy:', acc)



Epoch 1/5
8/8 [==============================] - 2s 3ms/step - loss: 0.2230 - accuracy: 0.2083
Epoch 2/5
8/8 [==============================] - 0s 2ms/step - loss: 0.2229 - accuracy: 0.1750
Epoch 3/5
8/8 [==============================] - 0s 2ms/step - loss: 0.2229 - accuracy: 0.2750
Epoch 4/5
8/8 [==============================] - 0s 2ms/step - loss: 0.2228 - accuracy: 0.2750
Epoch 5/5
8/8 [==============================] - 0s 2ms/step - loss: 0.2228 - accuracy: 0.2417
------------------------------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 124ms/step - loss: 0.2232 - accuracy: 0.2333
Mean Squared Error: 0.22323185205459595
Test accuracy: 0.23333333432674408


# Boston Housing Dataset with Keras


In [6]:
from keras.datasets import boston_housing

# we're going to use the Bosting housing dataset, but load it from keras 
housing_data = boston_housing.load_data(test_split=0.2) #it comes with it's own test/train split :) 
(X_train, Y_train), (X_test, Y_test) = housing_data


57026/57026 [==============================] - 0s 0us/step


In [7]:
# normalize the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [8]:
### need to execute this to get access to RSquare function
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 36.0 MB/s 


In [12]:
from keras import models
from keras import layers

from tensorflow_addons.metrics import RSquare

#build our model
n_features = X_train.shape[1] # get the number of input values for the input layer

model = Sequential(name='boston_housing1')   #initialize the model

#add some layers. Dense is a fully connected layer
model.add(layers.Dense(64, input_dim= n_features, activation='relu'))
model.add(layers.Dense(64, activation='relu'))

#Since we are doing a regressions, we only want one value as an ouput, so our last layer has a Dense layer with 1 neuron.
model.add(layers.Dense(1)) #default activation function is "linear"

from sklearn.metrics import r2_score
# Compile model
model.compile(loss='mean_squared_error',
              optimizer='sgd', 
              metrics= RSquare())
model.summary()

Model: "boston_housing1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                896       
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


In [17]:
# train the model
model.fit(X_train_norm, Y_train, epochs=100, batch_size=15, verbose=1)

print("----"*30)

# cross-validation; make predictions and get error
test_mse_score, test_r2_score = model.evaluate(X_test_norm, Y_test)
print('MSE:', test_mse_score)
print('r2:', test_r2_score)

Epoch 1/100
27/27 [==============================] - 0s 3ms/step - loss: 6.8083 - r_square: 0.9195
Epoch 2/100
27/27 [==============================] - 0s 3ms/step - loss: 6.9434 - r_square: 0.9179
Epoch 3/100
27/27 [==============================] - 0s 5ms/step - loss: 23.3588 - r_square: 0.7240
Epoch 4/100
27/27 [==============================] - 0s 5ms/step - loss: 12.3452 - r_square: 0.8541
Epoch 5/100
27/27 [==============================] - 0s 5ms/step - loss: 11.5629 - r_square: 0.8634
Epoch 6/100
27/27 [==============================] - 0s 4ms/step - loss: 11.2991 - r_square: 0.8665
Epoch 7/100
27/27 [==============================] - 0s 4ms/step - loss: 8.1543 - r_square: 0.9036
Epoch 8/100
27/27 [==============================] - 0s 4ms/step - loss: 7.5952 - r_square: 0.9102
Epoch 9/100
27/27 [==============================] - 0s 5ms/step - loss: 8.3635 - r_square: 0.9012
Epoch 10/100
27/27 [==============================] - 0s 5ms/step - loss: 8.1342 - r_square: 0.9039
Epoch

# In Class Exercise #2
What parameters from the models above do you think you can/should change? 


Try these
- Change the number of neruons in each layer.  
- Add a layer to the model.
- Change the activation function of the model, [here is the documentation](https://keras.io/api/layers/activations/)
- Change the optimizer, [here is the documentation](https://keras.io/api/optimizers/) with a list of options
- look at the [metrics](https://keras.io/api/metrics/) try adding another metric.


In [40]:
n_features = X_train.shape[1] # get the number of input values for the input layer

model = Sequential(name='boston_housing1')   #initialize the model

#add some layers. Dense is a fully connected layer
model.add(layers.Dense(64, input_dim= n_features, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='sigmoid'))

#Since we are doing a regressions, we only want one value as an ouput, so our last layer has a Dense layer with 1 neuron.
model.add(layers.Dense(1)) #default activation function is "linear"

from sklearn.metrics import r2_score
# Compile model
model.compile(loss='mean_squared_error',
              optimizer='sgd', 
              metrics= RSquare())
model.summary()

Model: "boston_housing1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_57 (Dense)            (None, 64)                896       
                                                                 
 dense_58 (Dense)            (None, 64)                4160      
                                                                 
 dense_59 (Dense)            (None, 64)                4160      
                                                                 
 dense_60 (Dense)            (None, 1)                 65        
                                                                 
Total params: 9,281
Trainable params: 9,281
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.fit(X_train_norm, Y_train, epochs=10, batch_size=15, verbose=1)

print("----"*30)

# cross-validation; make predictions and get error
test_mse_score, test_r2_score = model.evaluate(X_test_norm, Y_test)
print('MSE:', test_mse_score)
print('r2:', test_r2_score)

Epoch 1/10
27/27 [==============================] - 0s 3ms/step - loss: 80.7890 - r_square: 0.0453
Epoch 2/10
27/27 [==============================] - 0s 3ms/step - loss: 21.9155 - r_square: 0.7410
Epoch 3/10
27/27 [==============================] - 0s 3ms/step - loss: 16.4284 - r_square: 0.8059
Epoch 4/10
27/27 [==============================] - 0s 3ms/step - loss: 13.6637 - r_square: 0.8385
Epoch 5/10
27/27 [==============================] - 0s 3ms/step - loss: 15.1903 - r_square: 0.8205
Epoch 6/10
27/27 [==============================] - 0s 3ms/step - loss: 13.3400 - r_square: 0.8424
Epoch 7/10
27/27 [==============================] - 0s 3ms/step - loss: 12.3421 - r_square: 0.8542
Epoch 8/10
27/27 [==============================] - 0s 3ms/step - loss: 9.3261 - r_square: 0.8898
Epoch 9/10
27/27 [==============================] - 0s 3ms/step - loss: 9.7155 - r_square: 0.8852
Epoch 10/10
27/27 [==============================] - 0s 3ms/step - loss: 9.2360 - r_square: 0.8909
------------